In [35]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
import math
import os
import matplotlib.pyplot as plt
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import DataLoader,Dataset
from PIL import Image

In [ ]:
torch.manual_seed(42)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

In [ ]:
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)),
])
train_dataset = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True, num_workers=2)

In [ ]:
class Generator(nn.Module):
    def __init__(self, latent_dim, ngf, nc):
        super(Generator, self).__init__()
        self.main = nn.Sequential(
            nn.ConvTranspose2d(latent_dim, ngf * 8, 4, 1, 0, bias=False),
            nn.BatchNorm2d(ngf * 8),
            nn.ReLU(True),
            nn.ConvTranspose2d(ngf * 8, ngf * 4, 4, 2, 1, bias=False),
            nn.BatchNorm2d(ngf * 4),
            nn.ReLU(True),
            nn.ConvTranspose2d(ngf * 4, ngf * 2, 4, 2, 1, bias=False),
            nn.BatchNorm2d(ngf * 2),
            nn.ReLU(True),
            nn.ConvTranspose2d(ngf * 2, nc, 4, 2, 1, bias=False),
            nn.Tanh()
        )

    def forward(self, input):
        return self.main(input)

In [ ]:
class Discriminator(nn.Module):
    def __init__(self, nc, ndf):
        super(Discriminator, self).__init__()
        self.main = nn.Sequential(
            nn.Conv2d(nc, ndf, 4, 2, 1, bias=False),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Conv2d(ndf, ndf * 2, 4, 2, 1, bias=False),
            nn.BatchNorm2d(ndf * 2),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Conv2d(ndf * 2, ndf * 4, 4, 2, 1, bias=False),
            nn.BatchNorm2d(ndf * 4),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Conv2d(ndf * 4, 1, 4, 1, 0, bias=False),
            nn.Sigmoid()
        )

    def forward(self, input):
        return self.main(input)

In [ ]:
latent_dim = 100  # Size of the latent vector
ngf = 64  # Number of generator filters
ndf = 64  # Number of discriminator filters
nc = 3    # Number of channels in the input images (RGB)

generator = Generator(latent_dim, ngf, nc)
discriminator = Discriminator(nc, ndf)

In [34]:
criterion = nn.BCELoss()
optimizerG = optim.Adam(generator.parameters(), lr=0.0002, betas=(0.5, 0.999))
optimizerD = optim.Adam(discriminator.parameters(), lr=0.0002, betas=(0.5, 0.999))

num_epochs = 10 


In [30]:
loss_D=[]
loss_G=[]

for epoch in range(num_epochs):
    loss_each_D=[]
    loss_each_G=[]

    for i, data in enumerate(train_loader, 0):
        discriminator.zero_grad()
        real_data, _ = data
        batch_size = real_data.size(0)
        label = torch.full((batch_size,), 1.0)
        output = discriminator(real_data)
        errD_real = criterion(output.view(-1), label)
        errD_real.backward()
        noise = torch.randn(batch_size, latent_dim, 1, 1)
        fake_data = generator(noise)
        label.fill_(0.0)
        output = discriminator(fake_data.detach())
        errD_fake = criterion(output.view(-1), label)
        errD_fake.backward()

        errD=errD_fake+errD_real
        loss_each_D.append(errD.item())
        optimizerD.step()

        generator.zero_grad()
        label.fill_(1.0)  
        output = discriminator(fake_data)
        errG = criterion(output.view(-1), label)
        errG.backward()
        loss_each_G.append(errG.item())
        optimizerG.step()

        
        print("EPOCH",epoch,"Loss_D",errD.item(),"Loss_G",errG.item())
    
    loss_D.append(sum(loss_each_D)/len(loss_each_D))
    loss_G.append(sum(loss_each_G)/len(loss_each_G))

torch.save(generator.state_dict(), 'generator.pth')
torch.save(discriminator.state_dict(), 'discriminator.pth')


Files already downloaded and verified
[0/10][0/782] Loss_D: 1.4998 Loss_G: 1.4605
[0/10][100/782] Loss_D: 0.0766 Loss_G: 5.2175
[0/10][200/782] Loss_D: 0.6036 Loss_G: 2.9205
[0/10][300/782] Loss_D: 0.7869 Loss_G: 2.5276
[0/10][400/782] Loss_D: 0.7677 Loss_G: 2.2605
[0/10][500/782] Loss_D: 0.5334 Loss_G: 3.8287
[0/10][600/782] Loss_D: 0.7242 Loss_G: 2.4000
[0/10][700/782] Loss_D: 0.5402 Loss_G: 3.1009
[1/10][0/782] Loss_D: 1.0064 Loss_G: 2.0091
[1/10][100/782] Loss_D: 0.5122 Loss_G: 2.8126
[1/10][200/782] Loss_D: 0.4616 Loss_G: 2.7538
[1/10][300/782] Loss_D: 0.5534 Loss_G: 2.7803
[1/10][400/782] Loss_D: 0.7853 Loss_G: 3.9533
[1/10][500/782] Loss_D: 0.6132 Loss_G: 2.9005
[1/10][600/782] Loss_D: 0.2302 Loss_G: 2.8824
[1/10][700/782] Loss_D: 0.3411 Loss_G: 2.8650
[2/10][0/782] Loss_D: 0.1878 Loss_G: 2.9250
[2/10][100/782] Loss_D: 0.5352 Loss_G: 4.3931
[2/10][200/782] Loss_D: 0.3245 Loss_G: 3.1863
[2/10][300/782] Loss_D: 0.5533 Loss_G: 3.5962
[2/10][400/782] Loss_D: 0.6483 Loss_G: 4.5360
[2

In [ ]:
plt.figure(figsize=(10, 5))
plt.plot(loss_D, label='Discriminator Loss')
plt.plot(loss_G, label='Generator Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()
plt.show()

In [31]:

os.makedirs('images', exist_ok=True)
with torch.no_grad():
    for i in range(10):
        noise = torch.randn(1, latent_dim, 1, 1)
        fake = generator(noise).detach().cpu()
        plt.figure(figsize=(1,1))
        plt.axis("off")
        plt.imshow(np.transpose(fake[0],(1,2,0)))
        plt.savefig('images/fake_image_%d.png' % i)
        plt.close()

Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).
Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).
Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).
Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).
Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).
Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).
Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).
Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).
Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).
Clipping i

In [32]:
import torch
from torch import nn
from torch.autograd import Variable
from torch.nn import functional as F
import torch.utils.data

from torchvision.models.inception import inception_v3

import numpy as np
from scipy.stats import entropy

def inception_score(imgs, cuda=False, batch_size=32, resize=False, splits=1):
    """Computes the inception score of the generated images imgs

    imgs -- Torch dataset of (3xHxW) numpy images normalized in the range [-1, 1]
    cuda -- whether or not to run on GPU
    batch_size -- batch size for feeding into Inception v3
    splits -- number of splits
    """
    N = len(imgs)
    # print(imgs)

    assert batch_size > 0
    assert N > batch_size

    # Set up dtype
    if cuda:
        dtype = torch.cuda.FloatTensor
    else:
        if torch.cuda.is_available():
            print("WARNING: You have a CUDA device, so you should probably set cuda=True")
        dtype = torch.FloatTensor

    # Set up dataloader
    dataloader = torch.utils.data.DataLoader(imgs, batch_size=batch_size)

    # Load inception model
    inception_model = inception_v3(pretrained=True, transform_input=False).type(dtype)
    inception_model.eval();
    up = nn.Upsample(size=(299, 299), mode='bilinear').type(dtype)
    
    def get_pred(x):
        if resize:
            x = up(x)
        x = inception_model(x)
        return F.softmax(x).data.cpu().numpy()

    # Get predictions
    preds = np.zeros((N, 1000))

    for i, batch in enumerate(dataloader, 0):
        batch = batch.type(dtype)
        batchv = Variable(batch)
        batch_size_i = batch.size()[0]

        preds[i*batch_size:i*batch_size + batch_size_i] = get_pred(batchv)

    # Now compute the mean kl-div
    split_scores = []

    for k in range(splits):
        part = preds[k * (N // splits): (k+1) * (N // splits), :]
        py = np.mean(part, axis=0)
        scores = []
        for i in range(part.shape[0]):
            pyx = part[i, :]
            scores.append(entropy(pyx, py))
        split_scores.append(np.exp(np.mean(scores)))

    return np.mean(split_scores), np.std(split_scores)

# if __name__ == '__main__':
#     class IgnoreLabelDataset(torch.utils.data.Dataset):
#         def __init__(self, orig):
#             self.orig = orig

#         def __getitem__(self, index):
#             return self.orig[index][0]

#         def __len__(self):
#             return len(self.orig)

    # import torchvision.datasets as dset
    # import torchvision.transforms as transforms

    # cifar = dset.CIFAR10(root='data/', download=True,
    #                          transform=transforms.Compose([
    #                              transforms.Resize(32),
    #                              transforms.ToTensor(),
    #                              transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
    #                          ])
    # )

    # IgnoreLabelDataset(cifar)

    # print ("Calculating Inception Score...")
    # print (inception_score(IgnoreLabelDataset(cifar), cuda=False, batch_size=32, resize=True, splits=10))

In [33]:


def apply_transform_to_image(img_paths, transform):
    result=[]
    for img_path in img_paths:
        pil_image = Image.open(img_path)
        img_rgb = pil_image.convert('RGB')
        transformed_image = transform(img_rgb)
        result.append(transformed_image)
    return torch.stack(result)

# Example usage
image_paths = ['./images/fake_image_0.png', './images/fake_image_1.png', './images/fake_image_2.png', './images/fake_image_3.png','./images/fake_image_4.png', './images/fake_image_5.png', './images/fake_image_6.png', './images/fake_image_7.png', './images/fake_image_8.png', './images/fake_image_9.png']

# Define the transformation pipeline
transform = transforms.Compose([
    transforms.Resize(32),
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)),
])
transformed_images = apply_transform_to_image(image_paths,transform)

a,b=inception_score(transformed_images, cuda=False, batch_size=2, resize=True, splits=10)
print(a,b)


C:\Users\Nothing\AppData\Local\Temp\ipykernel_11852\1998552843.py:46: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  return F.softmax(x).data.cpu().numpy()


1.0 0.0
